## Pre-processing des data

1) Data cleaning

a) FBREF

In [18]:
import numpy as np
import pandas as pd
import pickle
import zlib

In [7]:
glossary = pd.read_csv("fbref_data\\Glossaire_FBREF.csv")
glossary.head()

,Unnamed: 0,0
0,Season,Season
1,Age,Age at season start
2,Country,Flag and name of the country.
3,Comp,Competition
4,LgRank,Squad finish in competition


In [13]:
glossary.describe()

glossary.rename(columns={"Unnamed: 0" : "Name", "0" : "Description"}, inplace = True)
glossary.head()

,Name,Description
0,Season,Season
1,Age,Age at season start
2,Country,Flag and name of the country.
3,Comp,Competition
4,LgRank,Squad finish in competition


In [ ]:
player_stats_fbref=pickle.loads(zlib.decompress(open('fbref_data/compressed_player_stats_fbref_1.bin','rb').read()))
player_stats_fbref.update(pickle.loads(zlib.decompress(open('fbref_data/compressed_player_stats_fbref_2.bin','rb').read())))
player_stats_fbref=pd.DataFrame(player_stats_fbref).transpose()
player_stats_fbref=player_stats_fbref.sort_index(axis=1,ascending=False)

teams_fbref=pickle.load(open('fbref_data/teams_fbref.pkl','rb'))

In [20]:
player_stats_fbref.head()

,meta,2024-2025,2024,2023-2024,2023,2022-2023,2022,2021-2022,2021,2020-2021,...,1999-2000,1999,1998-1999,1998,1997-1998,1996-1997,1995-1996,1994-1995,1993-1994,1992-1993
César Fernández,"{'height': '', 'weight': '', 'preferred foot':...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Yoel Lago,"{'height': '', 'weight': '', 'preferred foot':...","{'age': 20.0, 'team': 'Celta Vigo', 'country':...",NaN,"{'age': 19.0, 'team': 'Celta Vigo', 'country':...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Marc Vidal,"{'height': '', 'weight': '', 'preferred foot':...","{'age': 24.0, 'team': 'Celta Vigo', 'country':...",NaN,NaN,NaN,"{'age': 22.0, 'team': 'FC Andorra', 'country':...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Rui Silva,"{'height': 189.0, 'weight': 84.0, 'preferred f...","{'age': 30.0, 'team': 'Betis', 'country': 'es ...",NaN,"{'age': 29.0, 'team': 'Betis', 'country': 'es ...",NaN,"{'age': 28.0, 'team': 'Betis', 'country': 'es ...",NaN,"{'age': 27.0, 'team': 'Betis', 'country': 'es ...",NaN,"{'age': 26.0, 'team': 'Granada', 'country': 'e...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Abde Ezzalzouli,"{'height': 177.0, 'weight': 73.0, 'preferred f...","{'age': 22.0, 'team': 'Betis', 'country': 'es ...",NaN,"{'age': 21.0, 'team': 'Betis', 'country': 'es ...",NaN,"{'age': 20.0, 'team': 'Osasuna', 'country': 'e...",NaN,"{'age': 19.0, 'team': 'Barcelona', 'country': ...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


b)Sofascore

In [15]:
sf_data = pd.read_csv("sf_data\player_ratings.csv")
sf_data.head()

,Unnamed: 0,2024-2025,2023-2024,2022-2023,2021-2022,2020-2021,2019-2020,2018-2019,2017-2018,2016-2017,2015-2016
0,Bukayo Saka,8.04,7.36,5.97,7.67,6.43,6.04,6.25,NaN,NaN,NaN
1,Neto,9.41,7.31,4.00,7.71,7.09,6.52,8.29,7.84,8.20,8.0
2,Declan Rice,8.63,7.98,5.95,8.14,6.30,7.03,7.95,7.26,NaN,NaN
3,Gabriel Magalhães,6.70,7.86,6.00,6.66,5.56,6.68,6.00,7.12,5.89,NaN
4,Martin Ødegaard,6.95,7.14,8.25,8.36,5.79,6.51,NaN,NaN,NaN,NaN


c) Transfer market

In [68]:
player_stats_tm=pd.read_csv('tm_data/player_stats_tm.csv')
player_stats_tm.index=player_stats_tm.loc[:,'Unnamed: 0']
player_stats_tm=player_stats_tm.drop('Unnamed: 0',axis=1)

transfer_history=pickle.load(open('tm_data/transfer_history.pkl','rb'))


In [69]:

player_stats_tm.loc["Bernardo Silva"]["market_values"]

"{'2013': 600.0, '2014': 2500.0, '2015': 15000.0, '2016': 20000.0, '2017': 40000.0, '2018': 65000.0, '2019': 100000.0, '2020': 70000.0, '2021': 75000.0, '2022': 80000.0, '2023': 80000.0, '2024': 70000.0}"

In [70]:
def convertir_en_dictionnaire(chaine):
    chaine = chaine.strip('{}').replace(' ', '')
    paires = chaine.split(',')
    
    dictionnaire = {}
    for paire in paires:
        if ':' in paire: 
            cle, valeur = paire.split(':')
            cle = cle[1:-1] #pour envoyer les guillemets en plus sur l'année
            dictionnaire[int(cle)] = float(valeur)
    return dictionnaire

player_stats_tm['market_values']=player_stats_tm["market_values"].apply(convertir_en_dictionnaire)
player_stats_tm.head()

,positions,market_values
Unnamed: 0,,
Bernardo Silva,"['Attacking Midfield', 'Right Winger', 'Centra...","{2013: 600.0, 2014: 2500.0, 2015: 15000.0, 201..."
Ederson,['Goalkeeper'],"{2013: 300.0, 2014: 900.0, 2015: 1200.0, 2016:..."
Erling Haaland,['Centre-Forward'],"{2016: 200.0, 2017: 300.0, 2018: 5000.0, 2019:..."
Jack Grealish,"['Left Winger', 'Attacking Midfield', 'Left Mi...","{2013: 250.0, 2014: 1000.0, 2015: 4500.0, 2016..."
John Stones,"['Centre-Back', 'Right-Back', 'Defensive Midfi...","{2011: 50.0, 2012: 100.0, 2013: 3500.0, 2014: ..."


In [99]:
transfer_history["GB1"][2024]['AFC Bournemouth']['departures']

[['Kieffer Moore', 'Sheffield United', 1790.0],
 ['Jamal Lowe', 'Sheffield Wednesday', 'free transfer'],
 ['Hamed Junior Traoré', 'AJ Auxerre', 'loan transfer'],
 ['Chris Mepham', 'Sunderland AFC', 'loan transfer'],
 ['Neto', 'Arsenal FC', 'loan transfer'],
 ['Daniel Jebbison', 'Watford FC', 'loan transfer'],
 ['Emiliano Marcondes', 'Without ClubWithout Club', '-'],
 ['Darren Randolph', None, '-'],
 ['Luis Sinisterra', 'Leeds United', 'End of loanJun 30, 2024'],
 ['Andrei Radu', 'Inter Milan', 'End of loanJun 30, 2024'],
 ['Lloyd Kelly', 'Newcastle Utd', 'free transfer'],
 ['Romain Faivre', 'Paderborn 07', 'loan transfer'],
 ['Joe Rothwell', 'Leeds United', 'loan transfer'],
 ['Ryan Fredericks', 'Athletic Club', '-'],
 ['Kepa Arrizabalaga', 'Chelsea', 'End of loanMay 31, 2025'],
 ['Gavin Kilkenny', 'Luton Town', '?'],
 ['Alex Palmer', 'Atalanta', 'loan transfer'],
 ['Dominic Solanke', 'Tottenham', 64300.0],
 ['Enes Ünal', 'Getafe', 'End of loanJun 30, 2024'],
 ['Jaidon Anthony', 'Burnl

In [98]:
import geopandas as gpd
import pandas as pd
import folium
from folium import features
from ipywidgets import widgets, interactive
import json

# Charger les données
pl_clubs = pd.read_csv("pl_clubs_latlon.csv")

# Préparer un GeoDataFrame pour les clubs
gdf_clubs = gpd.GeoDataFrame(
    pl_clubs,
    geometry=gpd.points_from_xy(pl_clubs.longitude, pl_clubs.latitude),
    crs="EPSG:4326"
)

# Fonction pour calculer la valeur totale des joueurs d'un club
def calculate_market_value(player_stats, club_name):
    if club_name not in player_stats["club"].values:
        return 0
    players = player_stats[player_stats["club"] == club_name]
    return sum([max(player["market_values"].values(), default=0) for player in players.itertuples()])

# Ajouter une colonne pour la valeur totale
gdf_clubs["market_value"] = gdf_clubs["club_name"].apply(
    lambda club: calculate_market_value(player_stats_tm, club)
)

# Fonction pour afficher les arrivées et départs hors PL
def get_transfer_info(transfer_history, season, club):
    if season not in transfer_history["GB1"]:
        return [], []
    if club not in transfer_history["GB1"][season]:
        return [], []
    
    arrivals = transfer_history["GB1"].get(season, {}).get(club, {}).get("arrivals", [])
    departures = transfer_history["GB1"].get(season, {}).get(club, {}).get("departures", [])
    
    arrivals_outside = [a for a in arrivals if a[1] not in gdf_clubs["club_name"].values]
    departures_outside = [d for d in departures if d[1] not in gdf_clubs["club_name"].values]
    
    return arrivals_outside, departures_outside


def create_map(season, selected_club):
    m = folium.Map(location=[53.5, -2.3], zoom_start=6)

    for _, row in gdf_clubs.iterrows():
        folium.Circle(
            location=(row.latitude, row.longitude),
            radius=row.market_value / 1000,
            color="blue" if row.club_name == selected_club else "green",
            fill=True,
            fill_opacity=0.6,
            popup=f"<b>{row.club_name}</b><br>Valeur marchande : {row.market_value}"
        ).add_to(m)

    if selected_club and season in transfer_history["GB1"]:
        if selected_club in transfer_history["GB1"][season]:
            for season_data in transfer_history["GB1"][season][selected_club].values():
                for transfer in season_data:
                    target = gdf_clubs[gdf_clubs["club_name"] == transfer[1]]
                    if not target.empty:
                        folium.PolyLine(
                            locations=[
                                (gdf_clubs[gdf_clubs["club_name"] == selected_club].iloc[0].latitude, 
                                 gdf_clubs[gdf_clubs["club_name"] == selected_club].iloc[0].longitude),
                                (target.iloc[0].latitude, target.iloc[0].longitude)
                            ],
                            color="red" if transfer in transfer_history["GB1"].get(season, {}).get(selected_club, {}).get("arrivals", []) else "orange",
                            weight=2
                        ).add_to(m)

    return m

# Widgets pour l'interactivité
seasons = sorted(list(transfer_history["GB1"].keys()))
season_selector = widgets.Dropdown(options=seasons, description="Saison:")
club_selector = widgets.Dropdown(options=gdf_clubs["club_name"], description="Club:")

interactive_map = interactive(create_map, season=season_selector, selected_club=club_selector)
widgets.VBox([season_selector, club_selector, interactive_map])


KeyError: 'club'

In [128]:
from IPython.display import display
from folium.plugins import AntPath

def get_transfer_value(club,season):
    if club in transfer_history["GB1"][season].keys():
        transfer_fees = {'departures' : [], 'arrivals' : []}
        transfer_fees['arrivals'] = [fee[2]/1000 for fee in transfer_history["GB1"][season][club]["arrivals"] if type(fee[2])==float or type(fee[2])==int]
        for fee in transfer_history["GB1"][season][club]["departures"] :
            if type(fee[2])==float or type(fee[2])==int :
                transfer_fees['departures'].append(fee[2]/1000)
        return transfer_fees
    return {'departures' : [0], 'arrivals' : [0]}

data = pd.read_csv("pl_clubs_latlon.csv")

center_lat = data['latitude'].mean()
center_lon = data['longitude'].mean()

mymap = folium.Map(location=[center_lat, center_lon], zoom_start=6)
valeur = 4
radius=4
# Ajouter des points sur la carte
for _, row in data.iterrows():
    club = row['club_name']
    season = 2024
    transferts = get_transfer_value(club, season)
    valeur = round(sum(transferts['departures']) + sum(transferts['arrivals']))
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=radius,  # Rayon pondéré par la valeur marchande
        popup=f"{club}: {valeur}M€ \n Départs: {round(sum(transferts['departures']))}M€ \n Arrivées : {round(sum(transferts['arrivals']))}M€",
        color="blue",
        fill=True,
        fill_color="cyan",
        fill_opacity=0.6,
    ).add_to(mymap)
    if club in transfer_history["GB1"][season].keys():
        liste_departs = transfer_history["GB1"][season][club]["departures"]
        
        # Créer une couche spécifique pour les transferts depuis ce club
        layer = folium.FeatureGroup(name=f"Transferts depuis {club}")
        
        for transfert in liste_departs:
            if transfert[1] in list(data['club_name']):
                # Départ et arrivée
                depart = (
                    float(data[data['club_name'] == club]['latitude']),
                    float(data[data['club_name'] == club]['longitude']),
                )
                arrivee = (
                    float(data[data['club_name'] == transfert[1]]['latitude']),
                    float(data[data['club_name'] == transfert[1]]['longitude']),
                )
                
                # Ajouter une flèche au layer
                folium.PolyLine(
                    locations=[depart, arrivee],  # Liste des coordonnées
                    color="green",
                    weight=5,
                    opacity=0.8,
                ).add_to(layer)

                arrow = features.RegularPolygonMarker(
                    location=arrivee,  # Position de la pointe (destination)
                    color="black",
                    fill_color="green",
                    number_of_sides=3,  # Triangle pour la flèche
                    radius=10,          # Taille de la flèche
                    rotation=60         # Ajuste l'orientation
                )
                arrow.add_to(layer)

        # Ajouter la couche à la carte
        layer.add_to(mymap)

# Ajouter le contrôle des couches
folium.LayerControl(collapsed=False).add_to(mymap)

# Sauvegarder la carte
mymap.save("carte_interactive.html")
display(mymap)


C:\Users\massi\AppData\Local\Temp\ipykernel_24076\1755760911.py:47: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(data[data['club_name'] == club]['latitude']),
C:\Users\massi\AppData\Local\Temp\ipykernel_24076\1755760911.py:48: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(data[data['club_name'] == club]['longitude']),
C:\Users\massi\AppData\Local\Temp\ipykernel_24076\1755760911.py:51: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  float(data[data['club_name'] == transfert[1]]['latitude']),
C:\Users\massi\AppData\Local\Temp\ipykernel_24076\1755760911.py:52: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.il